In [1]:
import tkinter as tk
from tkinter import messagebox
from tkinter import filedialog

In [2]:
# store the task’s name, index, and prerequisite list
class Task:
    def __init__(self, name, index, prereq_numbers, duration):
        self.name = name
        self.index = index
        self.prereq_numbers = prereq_numbers
        self.duration = duration
    
    def __str__(self):
        return self.name
    
    def numbers_to_tasks(self, tasks):
        self.prereq_tasks = []
        for number in self.prereq_numbers:
            self.prereq_tasks.append(tasks[number])
            
    def set_times(self):
        self.start_time = 0
        for prereq in self.prereq_tasks:
            self.start_time = max(self.start_time, prereq.end_time)
        
        self.end_time = self.start_time + self.duration
    
    def mark_is_critical(self):
        self.is_critical = True
        for prereq in self.prereq_tasks:
            if prereq.end_time == self.start_time:
                prereq.mark_is_critical()

In [18]:
class PoSorter:
    def __init__(self):
        self.tasks = []
        
    def topo_sort(self):
        self.prepare_tasks()
        ready_tasks = []
        
        for task in self.tasks:
            if task.prereq_count == 0:
                ready_tasks.append(task)
        
        self.sorted_task = []
        while len(ready_tasks) > 0:
            ready_task = ready_tasks.pop(0)
            self.sorted_tasks.append(ready_task)

            for follower in ready_task.followers:
                follower.prereq_count -= 1
                if follower.prereq_count == 0:
                    ready_tasks.append(follower) # no prereqs, add it to the ready list
        
    def verify_sort(self):
        #verify the sort is valid
        for i in range(len(self.sorted_tasks)):
            task = self.sorted_tasks[i]
            for prereq in task.prereq_tasks:
                if self.sorted_tasks.index(prereq) >= i:
                    return f'Task [{prereq}] does not come before [{task}]'
        return f'Successfully sorted {len(self.sorted_tasks)} out of {len(self.tasks)} tasks.'
    
    def read_task(self, file):
        # read a task from the file
        # return a task or none if we reach the end of the file
        result = ''
        while len(result) == 0:
            line = file.readline()
            if len(line) == 0:
                return None
            
            line = line.strip()
            if len(line) > 0:
                fields = line.split(',', 3)
                index = int(fields[0])
                duration = int(fields[1])
                name = fields[2].strip()
                prereq_text = fields[3].replace('[', '').replace(']', '').strip()
                if len(prereq_text) == 0:
                    prereq_numbers = []
                else:
                    prereq_numbers = list(map(int, prereq_text.split(',')))
                return Task(name, index, prereq_numbers, duration)
    
    def load_po_file(self, filename):
        self.tasks = []
        file = open(filename, 'r')
        try:
            while True:
                new_task = self.read_task(file)
                if new_task == None:
                    break
                self.tasks.append(new_task)
        
        except Exception as e:
            messagebox.showinfo('Load Error', f'Error loading file {filename}.\n{e}')
        
        finally:
            file.close()
            
        for task in self.tasks:
            task.numbers_to_tasks(self.tasks)
            
    def prepare_tasks(self):
        for task in self.tasks:
            task.followers = []

        for task in self.tasks:
            for prereq in task.prereq_tasks:
                prereq.followers.append(task)
            task.prereq_count = len(task.prereq_tasks)
        
        for task in self.tasks:
            task.is_critical = False
    
    def build_pert_chart(self):
        self.prepare_tasks()
        
        # Move tasks with no prerequisites onto the ready list.
        ready_tasks = []
        for task in self.tasks:
            if task.prereq_count == 0:
                ready_tasks.append(task)

        self.sorted_tasks = [] #sorted task list
        self.columns = [] # columns list
        
        while len(ready_tasks) > 0:
            new_column = []
            self.columns.append(new_column)
            
            new_ready_tasks = []
            while len(ready_tasks) > 0:
                ready_task = ready_tasks.pop(0)
                new_column.append(ready_task)
                self.sorted_tasks.append(ready_task)
                ready_task.set_times()

                for follower in ready_task.followers:
                    follower.prereq_count -= 1
                    if follower.prereq_count == 0:
                        new_ready_tasks.append(follower) # no prereqs, add it to the new ready list
            
            # Mark the Finish task as critical.
            ready_tasks = new_ready_tasks
            
        self.tasks[len(self.tasks) - 1].mark_is_critical()
            
    def draw_pert_chart(self, canvas):
        canvas.delete('all')
        
        x_spacing = 40
        y_spacing = 10
        margin = 10
        cell_width = 50
        cell_height = 60
        
        for task in self.tasks:
            task.cell_bounds = None
        
        x = margin
        for column in self.columns:
            y = margin
            for task in column:
                task.cell_bounds = (x, y, x + cell_width, y + cell_height)
                y += cell_height + y_spacing
            x += cell_width + x_spacing
            
        for task in self.tasks:
            if task.cell_bounds == None:
                continue
            
            x1 = task.cell_bounds[0]
            y1 = (task.cell_bounds[1] + task.cell_bounds[3]) / 2
            
            for prereq in task.prereq_tasks:
                # Get the middle of the right edge of the prereq's cell.
                x2 = prereq.cell_bounds[2]
                y2 = (prereq.cell_bounds[1] + prereq.cell_bounds[3]) / 2
                
                width = 1
                fill = 'black'
                if prereq.end_time == task.start_time:
                    width = 3
                    if task.is_critical:
                        fill = 'red'
                canvas.create_line(x1, y1, x2, y2, arrow='first', width=width, fill=fill)
            
        for task in self.tasks:
            # Skip tasks that were not placed in the chart.
            if task.cell_bounds == None:
                continue
            
            rectangle_fill = 'lightblue1'
            text_fill = 'black'
            if task.is_critical:
                rectangle_fill = 'pink'
                text_fill = 'red'
                
            canvas.create_rectangle(*task.cell_bounds, fill=rectangle_fill, outline=text_fill)

            # Get the center of the task's cell.
            x = (task.cell_bounds[0] + task.cell_bounds[2]) / 2
            y = (task.cell_bounds[1] + task.cell_bounds[3]) / 2
            text = f'Task {task.index}\nDur: {task.duration}\nStart: {task.start_time}\nEnd: {task.end_time}'
            canvas.create_text(x, y, text=text, fill=text_fill, anchor='center', justify='center')

In [19]:
class App:
    # Create and manage the tkinter interface.
    def __init__(self):
        self.sorter = PoSorter()

        # Make the main interface.
        self.window = tk.Tk()
        self.window.title('draw_pert_chart')
        self.window.protocol('WM_DELETE_WINDOW', self.kill_callback)
        self.window.geometry('1000x550')

        # Build the menu.
        self.menubar = tk.Menu(self.window)
        self.menu_file = tk.Menu(self.menubar, tearoff=False)
        self.menu_file.add_command(label='Open...', command=self.open_po, accelerator='Ctrl+O')
        self.menu_file.add_separator()
        self.menu_file.add_command(label='Exit', command=self.kill_callback)
        self.menubar.add_cascade(label='File', menu=self.menu_file)
        self.window.config(menu=self.menubar)

        # Build the canvas.
        self.canvas = tk.Canvas(self.window, borderwidth=2, relief=tk.SUNKEN, bg='white')
        self.canvas.pack(padx=10, pady=(0, 10), fill=tk.BOTH, expand=True)

        self.window.bind('<Control-o>', self.ctrl_o_pressed)

        # Display the window.
        self.window.focus_force()
        self.window.mainloop()

    def kill_callback(self):
        self.window.destroy()

    def ctrl_o_pressed(self, event):
        self.open_po()
    def open_po(self):
        file_types = [('Partial Ordering', '*.po')]
        filename = filedialog.askopenfilename(defaultextension='.po', filetypes=file_types, initialdir='.', title='Open Partial Ordering')
        if not filename:
            return

        self.sorter.load_po_file(filename)
        self.sorter.build_pert_chart()
        self.sorter.draw_pert_chart(self.canvas)

    def sort(self):
        # Perform topological sorting.
        # No longer used.
        self.sorter.topo_sort()
        self.ordered_list.insert('end', *self.sorter.sorted_tasks)
        messagebox.showinfo('Sort Result', self.sorter.verify_sort())

In [20]:
App()